*Autor: Gabriel Mendes*   
[<img alt="Colaboratory logo" width="10%" src="https://img.shields.io/badge/LinkedIn-0077B5?style=for-the-badge&logo=linkedin&logoColor=white">](https://www.linkedin.com/in/gabriel-mendes-3b4a16149/)

### Limpeza de Dados
 
Neste notebook utilizei algumas funções da biblioteca Pandas para limpeza e tratamento de uma base de dados de empréstimo de clientes de um banco. Segue abaixo uma descrição de cada coluna do dataset:

    - ChaveSituacao: formado por:
        - Idade do cliente (a idade mínima para ser cliente é 18 anos)
        - Gênero do cliente:
            - M: Masculino
            - F: Feminino
        - Estado civil do cliente:
            - S: solteiro
            - C: casado
            - D: divorciado
            - V: viúvo
    - ClassRisco: formado por:
        - Classificação do cliente como (A,B,C) e indicador (+,- ou vazio)
        - Cor do cliente de acordo com um modelo de churn interno da empresa
    - CatCliente: formado por:
        - Categoria do cartão: qual o tipo de cartão do cliente:
            - Basic
            - Black
            - Platinum
        - Categoria VIP: categoria do cliente VIP (caso exista)
            - Alpha
            - Beta


In [72]:
import pandas as pd

In [73]:
base = pd.read_excel("ChavesClientes.xlsx", sheet_name = "base")

In [74]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento
0,1,32FC,Ccinza,Basic-Alpha,1
1,2,25MV,AAmarelo,Black,1
2,3,27MV,B-Amarelo,Basic-Beta,1
3,4,26FD,BAmarelo,Black,0
4,5,26FD,C-Amarelo,Black,0


In [75]:
base.groupby(["Pagamento","ChaveSituacao"])["Pagamento"].count()

Pagamento  ChaveSituacao
0          26FD             2
           28FC             2
           28MD             1
           30FC             1
           31MD             1
1          25FD             1
           25FV             1
           25MV             1
           26MC             2
           27MC             1
           27MD             2
           27MV             1
           28FS             1
           29MV             1
           31MV             1
           32FC             1
Name: Pagamento, dtype: int64

In [76]:
texto = "32FC"
texto[:2]

'32'

### Separando a coluna CatCliente com base em delimitadores de texto

Nesta etapa separo a coluna ChaveSituação em três colunas, com base na posição do texto.

In [77]:
base["Idade"] = base.ChaveSituacao.str[:2]
base["Genero"] = base.ChaveSituacao.str[2:3]
base["EstadoCivil"] = base["ChaveSituacao"].str[3:4]

In [78]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C
1,2,25MV,AAmarelo,Black,1,25,M,V
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V
3,4,26FD,BAmarelo,Black,0,26,F,D
4,5,26FD,C-Amarelo,Black,0,26,F,D


### Separando a coluna CatCliente com base em delimitadores de texto

Nesta etapa separo a coluna CatCliente em duas colunas, com base no delimitador "-".

In [79]:
base["CategoriaCartao"] = base.CatCliente.str.split("-").str.get(0)
base["CategoriaVIP"] = base.CatCliente.str.split("-").str.get(1)

In [80]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,CategoriaCartao,CategoriaVIP
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta
3,4,26FD,BAmarelo,Black,0,26,F,D,Black,NaN
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN


### Utilizando Regex para separar a coluna ClassRisco

Como não é possível separar a coluna ClassRisco utilizando apenas delimitadores ou pela posição do texto, utilizo Regex para a tarefa.

Regex (Regular Expressions) ou Expressões Regulares, é uma linguagem utilizada para encontrar padrões complexos em textos. As expressões regulares são formadas por uma combinação de caracteres literais que definem um conjunto de padrões a serem buscados.

Resumo de algumas funções utilizadas em Regex:

    - ^: O texto procurado começa com
    - $: Termina com
    - *: O último caracter do texto é repetido 0 ou mais vezes
    - +: O último caracter repetido 1 ou mais vezes
    - ?: O último caracter repetido 0 ou 1 vez
    - [A-Z]: qualquer valor em maiúsculo

In [81]:
import re

In [82]:
# Buscando Lucas no texto abaixo
re.findall('Lucas','meu nome é Lucas')

['Lucas']

In [83]:
# Verificando se começa ou termina com um caracter (usando ˆ, $)
re.findall('^meu','meu nome é Lucas')

['meu']

In [84]:
# Verificando se existe parte da palavra Lucas (usando *, +, ?)
re.findall('Lu?','meu nome é Lucas')

['Lu']

In [85]:
# Buscando por letras maiúsculas
re.findall('[A-Z]','meu nome é Lcas')

['L']

In [86]:
# Buscando agora a classificação do cliente dentro da coluna "ClassRisco"
re.findall("^[A-Z][^A-Z]?","B+amarelo")
#^[A-Z] : tudo que começa com maiusculo
#[A-Z] :  a segunda letra nao pode ser maiusculo

['B+']

In [87]:
base["Risco"] = base.ClassRisco.apply(lambda x: re.findall("^[A-Z][^A-Za-z]?",x)[0])

In [88]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,CategoriaCartao,CategoriaVIP,Risco
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha,C
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN,A
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta,B-
3,4,26FD,BAmarelo,Black,0,26,F,D,Black,NaN,B
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN,C-


### Transformando a coluna Idade em valores numéricos

Nesta etapa utilizo a função to_numeric para transformar em valores numéricos a coluna de Idade.

In [89]:
base["Idade"] = pd.to_numeric(base["Idade"]) 

In [90]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               20 non-null     int64 
 1   ChaveSituacao    20 non-null     object
 2   ClassRisco       20 non-null     object
 3   CatCliente       20 non-null     object
 4   Pagamento        20 non-null     int64 
 5   Idade            20 non-null     int64 
 6   Genero           20 non-null     object
 7   EstadoCivil      20 non-null     object
 8   CategoriaCartao  20 non-null     object
 9   CategoriaVIP     8 non-null      object
 10  Risco            20 non-null     object
dtypes: int64(3), object(8)
memory usage: 1.8+ KB


### Substituindo valores vazios na coluna CategoriaVIP

Nesta etapa substituo os valores NaN da coluna CategoriaVIP com a string "Comum".

In [91]:
base.loc[base.CategoriaVIP.isnull(),"CategoriaVIP"]="Comum"

In [92]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,CategoriaCartao,CategoriaVIP,Risco
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha,C
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,Comum,A
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta,B-
3,4,26FD,BAmarelo,Black,0,26,F,D,Black,Comum,B
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,Comum,C-


### Agrupando Valores com a função GroupBy

Por fim são feitos mais alguns agrupamentos do dataset, de forma que agora é possível verificar alguns padrões entre os tipos de clientes.

In [94]:
base.groupby(["Pagamento","CategoriaCartao"])["Pagamento"].count()

Pagamento  CategoriaCartao
0          Basic              3
           Black              3
           Platinum           1
1          Basic              3
           Black              4
           Platinum           6
Name: Pagamento, dtype: int64

É possível verificar que maioria dos clientes com cartão Platinum são bons pagadores (coluna pagamento com valor igual 1).

In [95]:
base.groupby(["Pagamento","CategoriaVIP"])["Pagamento"].count()

Pagamento  CategoriaVIP
0          Alpha           1
           Beta            2
           Comum           4
1          Alpha           3
           Beta            2
           Comum           8
Name: Pagamento, dtype: int64

Observando a coluna CategoriaVIP, conclui-se que os clientes do tipo Comum são os melhores pagadores.

In [96]:
base.groupby(["Pagamento","Risco"])["Pagamento"].count()

Pagamento  Risco
0          B        2
           C-       5
1          A        3
           A-       4
           B-       2
           C        3
           C+       1
Name: Pagamento, dtype: int64

Ja verificando a coluna Risco, observa-se que apenas os clientes de risco B e C- são inadimplentes (coluna Pagamento = 0). 